# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [1]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$


In [2]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)
    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
    import itertools
    for text in train_text:
        words = f'<s> {text.strip()} </s>'.split()
        if not bos:
            words = words[1:]
        if not eos:
            words = words[:-1]
        for step in range(1, order + 1):
            for i in range(len(words) - (step - 1)):
                ngram = tuple(words[i:i + step])
                if ngram not in ngrams:
                    ngrams[ngram] = 1
                else:
                    ngrams[ngram] += 1
        

    # 
    return dict(ngrams)

In [3]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.

In [4]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    V = len([ngram for ngram in counts.keys() if len(ngram) == 1])
    prob = None
    count_sub_grams = None
    
    count = counts[ngram] if ngram in counts.keys() else 0

    if len(ngram) == 1: 
        count_sub_grams = V
    else:
        sub_ngram = ngram[:-1]
        count_sub_grams = counts[sub_ngram] if sub_ngram in counts.keys() else 0

    try:
        prob = (count + k) / (count_sub_grams + k*V)
    except ZeroDivisionError:
        prob = 0
    # TODO
    return prob

In [5]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/12
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [6]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = bos
        self.bos = eos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        # TODO
        # Подсчет vocab и ngrams_count по обучающей выборке
        
        self.ngrams_count = dict()
        for text in train_text:
            words = f'<s> {text.strip()} </s>'.split()
            if not self.bos:
                words = words[1:]
            if not self.eos:
                words = words[:-1]
            for step in range(1, self.order + 1):
                for i in range(len(words) - (step - 1)):
                    ngram = tuple(words[i:i + step])
                    if ngram not in self.ngrams_count:
                        self.ngrams_count[ngram] = 1
                    else:
                        self.ngrams_count[ngram] += 1

            
        if not self.vocab:
            self.vocab = [ngram for ngram in self.ngrams_count.keys() if len(ngram) == 1]
                
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO 
        # считаем логарифм вероятности конкретной нграмы
        prob = None
        count_sub_grams = None

        count = self.ngrams_count[ngram] if ngram in self.ngrams_count.keys() else 0

        if len(ngram) == 1: 
            count_sub_grams = self.V
        else:
            sub_ngram = ngram[:-1]
            count_sub_grams = self.ngrams_count[sub_ngram] if sub_ngram in self.ngrams_count.keys() else 0

        try:
            prob = (count + self.k) / (count_sub_grams + self.k*self.V)
        except ZeroDivisionError:
            prob = 0
        
        return np.log(prob)
           
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        
        # TODO 
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        logprob = 0
        n_probs = 0

        # log( (<s>,) ) + log( (<s>, word) ) + log( (word, word2) ) + ... + log( (word9, </s>) ) + log( (</s>,) ) # I think at the beginning
        # log( (<s>,) ) + log( (<s>, word) ) + log( (word, word2) ) + ... + log( (word9, </s>) ) # It turned out to be
        for i in range(0, len(words)):
            if i + 1 < self.order:
                logprob += self.predict_ngram_log_proba(tuple(words[:i+1]))
            else:
                start = i + 1 - self.order
                end = i + 1
                logprob += self.predict_ngram_log_proba(tuple(words[start:end]))
            n_probs += 1

        return logprob / n_probs
        
    def ppl(self, text: List[str]) -> float:
        #TODO 
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        perplexity = 0

        for sentence in text:
            words = sentence.split()
            perplexity += np.exp( -1 * self.predict_log_proba(words=words))

        return perplexity / len(text)

In [7]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/28 * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/28 * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 6.99, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (4 балла)

In [9]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix
    
    if topk < 1:
        raise Exception('topk argument is less than 1')

    perplexities = dict()

    for word in lm.vocab:
        if word[0] == '<s>' or word[0] == '</s>':
            continue

        hyp = f"{' '.join(prefix)} {word[0]}"

        # print(hyp)
        
        perplexities[word[0]] = lm.ppl( [hyp] )
    
    word = sorted(perplexities.items(), key=lambda x: x[1])[:topk]

    return word

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

In [12]:
train_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени",
              "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
              "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
              "мы сэкономим уйму времени если я сейчас сойду с ума по-моему",
              'привет привет как дела',
              'практическое сентября',
              'второе практическое занятие пройдет в офлайне тридцать второго сентября в двенадцать часов тридцать две минуты минуты',
              'в офлайне в тридцать два двенадцать'
              ]
for prefix in [['по-моему', 'сэкономим', 'нграм', 'вероятности'], ['по-моему'], ['сэкономим'], ['нграм'], ['вероятности']]:
    print('-'*5, f'{prefix=}', '-'*5)
    for i in range(1, 5):
        lm = NgramLM(order=i, bos=True, eos=True, k=1, predefined_vocab=None)
        lm.fit(train_data)

        print(f'order = {i}', predict_next_word(lm=lm, prefix=prefix, topk=4))

----- prefix=['по-моему', 'сэкономим', 'нграм', 'вероятности'] -----
order = 1 [('по-моему', 16.60322293202068), ('если', 16.60322293202068), ('я', 16.60322293202068), ('сейчас', 16.60322293202068)]
order = 2 [('времени', 19.88643665600155), ('дела', 20.804301624473556), ('сентября', 20.89595763002353), ('двенадцать', 20.89595763002353)]
order = 3 [('по-моему', 22.28474181317736), ('если', 22.28474181317736), ('я', 22.28474181317736), ('прямо', 22.28474181317736)]
order = 4 [('по-моему', 22.28474181317736), ('если', 22.28474181317736), ('я', 22.28474181317736), ('прямо', 22.28474181317736)]
----- prefix=['по-моему'] -----
order = 1 [('по-моему', 9.242414306999132), ('если', 9.242414306999132), ('я', 9.242414306999132), ('сейчас', 9.242414306999132)]
order = 2 [('мы', 13.828556281358088), ('времени', 13.828556281358088), ('дела', 14.96477322091469), ('сентября', 15.080339896652132)]
order = 3 [('если', 14.74625072323357), ('мы', 14.860129841247415), ('по-моему', 17.39770816309208), ('я'